# IBM Ponder This - April 2024

## Problem Statement

The Tower of Hanoi game consists of three rods and n disks of different sizes (denoted by $1,2,\ldots,n$) that can slide onto any rod. The initial state of the game consists of all the disks on one rod in ascending order, with disk 1 , the smallest and lightest of the disks on top.

Each move in the game consists of choosing one rod, picking up the top disk from that rod, and placing it on another rod. Such a move is allowed only if the disk being moved is smaller than the top disk on the target rod, or if the target rod is empty.

If the three rods are placed on different points on a circle, at any given state, there are three possible moves:
0. Take disk "1" from its rod and move it clockwise to another rod.
1. Take disk "1" from its rod and move it counterclockwise to another rod.
2. Take a disk which is not "1" and move it to another rod.

Move 2 will not always be available, such as when all the disks are currently located on one rod. In such instances, we say that Move 2 does nothing. In all other cases, there is only one legal way to perform Move 2.

We can describe the flow of games using a string describing moves. For example, "0202020", when performed on the initial state of a game with $n=3$ , moves all the disks one rod clockwise. We call the state where all the disks are on the clockwise-located rod the winning state of the game. Another way to describe that game is with the string "02" which must be performed for exactly 7 steps, with the convention that once the end of the string is reached, we start over at the beginning. It can be shown that the $n$-disk game is won by using the string "02" for $2^n-1$ steps with $n$ odd, and "12" for $2^n-1$ steps with $n$ even.

For example, when $n=3$ and the move string is "0202112", after 8 steps the winning step is reached, and the next step does nothing, so we say the winning state is reached both on step 8 and step 9. Similarly, for $n=4$ and move string "200211", the winning state is reached after 41 and 122 moves.

However, if we start two simultaneous games, one with $n=3$ and "0202112", and the other with $n=4$ and "200211", and perform each step in both games at the same time, both games will reach a winning state together for the first time after 932 steps.

Your Goal: For the two games defined by $n=7$ and "12021121120020211202121", and $n=10$ and "0211202112002", find the minimal number of steps at which both games reach a winning state at the same time.

A Bonus "*" will be given for finding the minimal number of steps such that both the above games and also the game defined by $n=9$ and "20202020021212121121202120200202002121120202
112021120020021120211211202002112021120211200212112020212120211" reach a winning state for the first time.

## Solution

We create a function to simulate the game and count the number of steps between consecutive winning positions. Then, we find the smallest repeating sequence of number of moves between 2 winning positions. Once we have this for several games, we can simply generate all the winning states for all the games and find the first one where they all match.

In [1]:
import time

def get_new_pos(rods, n, pos, move):
    if move == '0':
        if pos == 0:
            rods[1].append(rods[0].pop())
            pos = 1
        elif pos == 1:
            rods[2].append(rods[1].pop())
            pos = 2
        else:
            rods[0].append(rods[2].pop())
            pos = 0
    elif move == '1':
        if pos == 0:
            rods[2].append(rods[0].pop())
            pos = 2
        elif pos == 1:
            rods[0].append(rods[1].pop())
            pos = 0
        else:
            rods[1].append(rods[2].pop())
            pos = 1
    elif len(rods[0]) != n and len(rods[1]) != n and len(rods[2]) != n:
        if pos == 0:
            if len(rods[1]) == 0:
                rods[1].append(rods[2].pop())
            elif len(rods[2]) == 0:
                rods[2].append(rods[1].pop())
            elif rods[1][-1] < rods[2][-1]:
                rods[2].append(rods[1].pop())
            else:
                rods[1].append(rods[2].pop())
        elif pos == 1:
            if len(rods[0]) == 0:
                rods[0].append(rods[2].pop())
            elif len(rods[2]) == 0:
                rods[2].append(rods[0].pop())
            elif rods[0][-1] < rods[2][-1]:
                rods[2].append(rods[0].pop())
            else:
                rods[0].append(rods[2].pop())
        if pos == 2:
            if len(rods[1]) == 0:
                rods[1].append(rods[0].pop())
            elif len(rods[0]) == 0:
                rods[0].append(rods[1].pop())
            elif rods[1][-1] < rods[0][-1]:
                rods[0].append(rods[1].pop())
            else:
                rods[1].append(rods[0].pop())
    return rods, pos


def get_repeating_sequence(n, sequence, run_time):
    """ Simulate the game and find smallest repeating subsequence of number of moves between winning states """
    rods = [[i for i in range(1, n + 1)][::-1], [], []]
    total_count = 0
    i = 0
    pos1 = 0
    res = []
    start_time = time.time()
    # Simulate the game for some time
    while time.time() < start_time + run_time:
        if i == len(sequence):
            i = 0
        move = sequence[i]
        rods, pos1 = get_new_pos(rods, n, pos1, move)
        i += 1
        total_count += 1
        if len(rods[1]) == n:
            res.append(total_count)
    # Comute the number of moves between consecutive winning states
    diff = []
    for i in range(1, len(res)):
        diff.append(res[i] - res[i - 1])
    # Find smallest repeating sequence
    seq = find_smallest_repeating_sequence(diff)
    a = res[0]
    return a, seq


def sequence_repeats(subsequence, lst):
    """Check if a given subsequence repeats in the list, allowing for incomplete
    repetition at the end."""
    n = len(subsequence)
    for i in range(0, len(lst), n):
        # Handle incomplete pattern at the end
        if i + n > len(lst):
            if subsequence[:len(lst)-i] != lst[i:]:
                return False
        else:
            if subsequence != lst[i:i+n]:
                return False
    return True


def find_smallest_repeating_sequence(lst):
    smallest_len = 20
    smallest_seq = []
    for seq_len in range(1, 20):  # Iterate through all possible lengths
        for i in range(len(lst) - seq_len + 1):
            current_seq = lst[i:i + seq_len]
            if sequence_repeats(current_seq, lst):
                if seq_len < smallest_len:
                    smallest_len = seq_len
                    smallest_seq = current_seq
                    break  # Found the smallest repeating sequence so far, no need to continue
        if smallest_seq:  # If we've found a sequence, no need to check longer lengths
            break
    return smallest_seq


def find_matching_sequence(a1, a2, seq1, seq2):
    list1 = [a1]
    list2 = [a2]
    i = 0
    j = 0
    start_time = time.time()
    while time.time() < start_time + 20:
        if i == len(seq1):
            i = 0
        list1.append(list1[-1] + seq1[i])
        i += 1
        if j == len(seq2):
            j = 0
        list2.append(list2[-1] + seq2[j])
        j += 1

    res = []
    i, j = 0, 0
    while i < len(list1) and j < len(list2):
        if list1[i] == list2[j]:
            res.append(list1[i])
            i += 1
            j += 1
        elif list1[i] < list2[j]:
            i += 1
        else:
            j += 1
    diff = []
    for i in range(1, len(res)):
        diff.append(res[i] - res[i - 1])

    return res[0], find_smallest_repeating_sequence(diff)

In [2]:
n = 7
sequence = "12021121120020211202121"

a1, seq1 = get_repeating_sequence(n, sequence, 10)

print(a1)
print(seq1)

1404
[1, 2798, 1, 8378]


In [3]:
n = 10
sequence = "0211202112002"

a2, seq2 = get_repeating_sequence(n, sequence, 10)

print(a2)
print(seq2)

4429
[1, 6655]


In [4]:
n = 9
sequence = "20202020021212121121202120200202002121120202112021120020021120211211202002112021120211200212112020212120211"

a3, seq3 = get_repeating_sequence(n, sequence, 10)

print(a3)
print(seq3)

86943
[8, 718, 1, 58204, 23, 267, 1, 67, 1, 9, 9, 1, 174700]


In [5]:
a12, seq12 = find_matching_sequence(a1, a2, seq1, seq2)
print(f'Answer for 2 games is {a12}.')

Answer for 2 games is 16511310.


In [6]:
a123, seq123 = find_matching_sequence(a12, a3, seq12, seq3)
print(f'Answer for 3 games is {a123}.')

Answer for 3 games is 1169723214.


In [7]:
# Other method to match the three lists directly without using the intermediary sequence

def find_lowest_matching_number(list1, list2, list3):
    i, j, k = 0, 0, 0
    while i < len(list1) and j < len(list2) and k < len(list3):
        # Check if all three elements are the same
        if list1[i] == list2[j] == list3[k]:
            return list1[i]

        # Find the smallest element and increment the respective pointer
        min_val = min(list1[i], list2[j], list3[k])
        if list1[i] == min_val:
            i += 1
        if list2[j] == min_val:
            j += 1
        if list3[k] == min_val:
            k += 1
    return None

list1 = [a1]
list2 = [a2]
list3 = [a3]
i = 0
j = 0
k = 0
start_time = time.time()
while time.time() < start_time + 20:
    if i == len(seq1):
        i = 0
    list1.append(list1[-1] + seq1[i])
    i += 1
    if j == len(seq2):
        j = 0
    list2.append(list2[-1] + seq2[j])
    j += 1
    if k == len(seq3):
        k = 0
    list3.append(list3[-1] + seq3[k])
    k += 1

find_lowest_matching_number(list1, list2, list3)

1169723214